In [18]:
import numpy as np
import pandas as pd
import csv

# text prepocessing
from preprocessor.api import clean as p
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [19]:
# Import dataset
df = pd.read_csv('../Data/1. Raw/Dr_Richard_Lee_MARS_Sirup_Gagal_Ginjal_Komentar.csv')

# Menampilkan total dataset
print("Total Dataset :", len(df))

df.head()

Total Dataset : 4804


,videoId,textDisplay,textOriginal,authorDisplayName,authorProfileImageUrl,authorChannelUrl,authorChannelId,canRate,viewerRating,likeCount,publishedAt,updatedAt,parentId,commentId
0,video1,Baiknya bagaimana menurut kalian?,Baiknya bagaimana menurut kalian?,"dr. Richard Lee, MARS",https://yt3.ggpht.com/ytc/AL5GRJVsyV8johIkR8ZI...,http://www.youtube.com/channel/UCMgMzUVQjzkEqH...,{'value': 'UCMgMzUVQjzkEqHmJmuXR0Hw'},True,none,428,2022-10-21T11:14:34Z,2022-10-21T11:14:34Z,NaN,UgxxMfyEs8p9QKFp-nZ4AaABAg
1,video1,Betul dok betul dok,Betul dok betul dok,Sri Lestari,https://yt3.ggpht.com/ytc/AL5GRJVvEStECt3rwMjI...,http://www.youtube.com/channel/UCvNy9mPPeEoReF...,{'value': 'UCvNy9mPPeEoReFLzd4A0iWQ'},True,none,0,2023-03-20T23:56:33Z,2023-03-20T23:56:33Z,NaN,UgxbHv_nznpmjiu3eZR4AaABAg
2,video1,Betul dok mungkin efek sa di,Betul dok mungkin efek sa di,Sri Lestari,https://yt3.ggpht.com/ytc/AL5GRJVvEStECt3rwMjI...,http://www.youtube.com/channel/UCvNy9mPPeEoReF...,{'value': 'UCvNy9mPPeEoReFLzd4A0iWQ'},True,none,0,2023-03-20T23:31:32Z,2023-03-20T23:31:32Z,NaN,UgzbeyPlGYzilX0MLaR4AaABAg
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.","Bpom aneh, tidak kerja. Kenapa ko seperti itu.",Canliwmarang Marang,https://yt3.ggpht.com/ytc/AL5GRJVkYdevq5Pot_-z...,http://www.youtube.com/channel/UC3dR3sO99n3P8U...,{'value': 'UC3dR3sO99n3P8UksFr-NHMQ'},True,none,0,2023-03-08T22:14:23Z,2023-03-08T22:14:23Z,NaN,UgxmYRPRwYLEInorTIZ4AaABAg
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,Benar banget anak sy aj hrs sirup minum obatny...,Citra Lia,https://yt3.ggpht.com/ytc/AL5GRJWrdTmL16NKJ0ed...,http://www.youtube.com/channel/UCO_5xbHx4mHjJe...,{'value': 'UCO_5xbHx4mHjJedjz9iiqUQ'},True,none,0,2023-03-08T00:29:54Z,2023-03-08T00:29:54Z,NaN,UgxQL8hr2QMy9Lq31yZ4AaABAg


In [20]:
# Rename columns 
new_column_names = {'videoId': 'video_id', 'textOriginal': 'comment_text'}
df = df.rename(columns=new_column_names)

# Keep only the 'video_id' and 'comment_text' columns
df = df[['video_id', 'comment_text']]

df.head()

,video_id,comment_text
0,video1,Baiknya bagaimana menurut kalian?
1,video1,Betul dok betul dok
2,video1,Betul dok mungkin efek sa di
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu."
4,video1,Benar banget anak sy aj hrs sirup minum obatny...


In [21]:
# Cleaning
def text_cleaning(text):
    # case folding
    text = text.lower()
    # remove mention YouTube
    text = re.sub(r'@[A-Za-z0-9_]+\s[A-Za-z0-9_]+(\s[A-Za-z0-9_]+)?', '', text)
    # remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # remove punctuation, numbers, and extra whitespace
    text = re.sub(r'[^\w\s]|_', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split())
    # remove emoji and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    return text

In [22]:
df["text_cleaned"] = df["comment_text"].apply(text_cleaning)
df.head(10)

,video_id,comment_text,text_cleaned
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian
1,video1,Betul dok betul dok,betul dok betul dok
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...
5,video1,Makasih ilmunya dokter semoga berkah🥰🥰🥰🥰,makasih ilmunya dokter semoga berkah
6,video1,Dok tempra gimana? Soalnya anakku kalau demam ...,dok tempra gimana soalnya anakku kalau demam d...
7,video1,BIOSOLFON KIDS AMAN GAK KAK,biosolfon kids aman gak kak
8,video1,apa kurang ilmu ya dok,apa kurang ilmu ya dok
9,video1,Cara media massa cari uang.... Memainkan keres...,cara media massa cari uang memainkan keresahan...


In [23]:
# Filtering (Stopword)
def stopword_removal(text):
    # Membuat StopWordRemoverFactory dari pustaka Sastrawi
    factory = StopWordRemoverFactory()
    # Membuat objek stopword remover
    stopword = factory.create_stop_word_remover()  
    # Menghapus kata-kata tidak penting dari teks dan mengembalikan teks yang telah difilter
    return stopword.remove(text)

# Stemming
def stemming(text):
    # Membuat StemmerFactory dari pustaka Sastrawi
    factory = StemmerFactory()
    # Membuat objek stemmer
    stemmer = factory.create_stemmer()  
    # Melakukan stemming pada teks dan mengembalikan teks yang telah di-stem
    text = stemmer.stem(text)  
    
    return text

In [24]:
df["text_removal"] = df["text_cleaned"].apply(lambda x: stopword_removal(stemming(x)))
df.head(10)

,video_id,comment_text,text_cleaned,text_removal
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...
5,video1,Makasih ilmunya dokter semoga berkah🥰🥰🥰🥰,makasih ilmunya dokter semoga berkah,makasih ilmu dokter moga berkah
6,video1,Dok tempra gimana? Soalnya anakku kalau demam ...,dok tempra gimana soalnya anakku kalau demam d...,dok tempra gimana soal anak kalau demam kasih ...
7,video1,BIOSOLFON KIDS AMAN GAK KAK,biosolfon kids aman gak kak,biosolfon kids aman gak kak
8,video1,apa kurang ilmu ya dok,apa kurang ilmu ya dok,apa kurang ilmu dok
9,video1,Cara media massa cari uang.... Memainkan keres...,cara media massa cari uang memainkan keresahan...,cara media massa cari uang main resah bingung ...


In [25]:
# Bahasa alay removal
# Mengimpor kamus alay pertama dari repositori 'analisis-sentimen'
kamus_alay_1 = pd.read_csv(
    "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/kbba.txt",
    delimiter="\t",  # Menggunakan tab sebagai pemisah
    header=None,  # Tidak ada baris header dalam file
    names=['slang', 'formal'])  # Memberi nama kolom dalam DataFrame

# Mengimpor kamus alay kedua dari repositori 'kamus-alay'
kamus_alay_2 = pd.read_csv(
    "https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv",
    usecols=["slang", "formal"])  # Memilih kolom "slang" dan "formal" saja

# Menggabungkan kedua kamus alay menjadi satu DataFrame
kamus_alay = pd.concat([kamus_alay_1, kamus_alay_2])  # Menggabungkan kedua DataFrame

In [26]:
# Membuat kamus kosong untuk menyimpan pasangan kata slang dan kata formal
dict_alay = dict()

# Iterasi melalui setiap baris dalam DataFrame kamus_alay
for index, row in kamus_alay.iterrows():
    # Menambahkan entri kata slang dan kata formal ke dalam dict_alay
    dict_alay[row['slang']] = row['formal']

# Fungsi untuk menggantikan kata slang dengan kata formal
def normalize_text(text):
    # Tokenisasi teks menjadi list kata
    word_tokens = word_tokenize(text)
    # Menggantikan kata slang dengan kata formal menggunakan dict_alay (jika ada)
    result = [dict_alay.get(w, w) for w in word_tokens]
    # Menggabungkan kata-kata yang telah dinormalisasi menjadi string
    return ' '.join(result)

In [27]:
df["text_normalized"] = df["text_removal"].apply(lambda x: normalize_text(x))
df.head(10)

,video_id,comment_text,text_cleaned,text_removal,text_normalized
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa,betul dok mungkin efek sa
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu,bpom aneh kerja kok itu
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...,benar banget anak saya saja harus sirup minum ...
5,video1,Makasih ilmunya dokter semoga berkah🥰🥰🥰🥰,makasih ilmunya dokter semoga berkah,makasih ilmu dokter moga berkah,terima kasih ilmu dokter moga berkah
6,video1,Dok tempra gimana? Soalnya anakku kalau demam ...,dok tempra gimana soalnya anakku kalau demam d...,dok tempra gimana soal anak kalau demam kasih ...,dok tempra bagaimana soal anak kalau demam kas...
7,video1,BIOSOLFON KIDS AMAN GAK KAK,biosolfon kids aman gak kak,biosolfon kids aman gak kak,biosolfon kids aman enggak kak
8,video1,apa kurang ilmu ya dok,apa kurang ilmu ya dok,apa kurang ilmu dok,apa kurang ilmu dok
9,video1,Cara media massa cari uang.... Memainkan keres...,cara media massa cari uang memainkan keresahan...,cara media massa cari uang main resah bingung ...,cara media massa cari uang main resah bingung ...


In [28]:
# Membuat kamus untuk menggantikan kata-kata dalam teks
dict_clean = {
    "gin": "gini", "ya": "iya", "yaa": "iya", "gue": "aku", "saya": "aku", "moga": "semoga", "lo": "kamu",
    "gw": "aku", "djual": "dijual", "lg": "lagi", "tp": "tapi", "jd": "jadi", "brg": "barang", "blm": "belum",
    "sm": "sama", "km": "kamu", "dg": "dengan", "jg": "juga", "dlm": "dalam", "hrs": "harus", "bs": "bisa",
    "skrg": "sekarang", "jln": "jalan", "pny": "punya", "brp": "berapa", "ngg": "nggak", "ga": "tidak",
    "bkn": "bukan", "krn": "karena", "org": "orang", "thn": "tahun", "byr": "bayar", "kmrn": "kemarin",
    "bg": "bang", "tks": "terima kasih", "trm": "terima", "bolhkh" : "bolehkah", "ktakutan" : "ketakutan", "kslmtn" : "keselamatan",
    "untkk" : "untuk",
}

# Fungsi untuk menggantikan kata-kata dalam teks berdasarkan kamus dict_clean
def replace_word(data):
    # Tokenisasi teks menjadi list kata
    word_tokens = word_tokenize(data)
    # Menggantikan kata dengan kata yang sesuai dari dict_clean (jika ada)
    result = [dict_clean.get(w, w) for w in word_tokens]
    # Menggabungkan kata-kata yang telah digantikan menjadi string
    return ' '.join(result)

# Menambahkan kolom baru 'text_manual_replaced' yang berisi teks dengan kata-kata digantikan
normalized_col_index = df.columns.get_loc("text_normalized")
df.insert(normalized_col_index + 1, "text_manual_replaced", df["text_normalized"].apply(lambda x: replace_word(x)))

df.head(10)

,video_id,comment_text,text_cleaned,text_removal,text_normalized,text_manual_replaced
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa,betul dok mungkin efek sa,betul dok mungkin efek sa
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu,bpom aneh kerja kok itu,bpom aneh kerja kok itu
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...,benar banget anak saya saja harus sirup minum ...,benar banget anak aku saja harus sirup minum o...
5,video1,Makasih ilmunya dokter semoga berkah🥰🥰🥰🥰,makasih ilmunya dokter semoga berkah,makasih ilmu dokter moga berkah,terima kasih ilmu dokter moga berkah,terima kasih ilmu dokter semoga berkah
6,video1,Dok tempra gimana? Soalnya anakku kalau demam ...,dok tempra gimana soalnya anakku kalau demam d...,dok tempra gimana soal anak kalau demam kasih ...,dok tempra bagaimana soal anak kalau demam kas...,dok tempra bagaimana soal anak kalau demam kas...
7,video1,BIOSOLFON KIDS AMAN GAK KAK,biosolfon kids aman gak kak,biosolfon kids aman gak kak,biosolfon kids aman enggak kak,biosolfon kids aman enggak kak
8,video1,apa kurang ilmu ya dok,apa kurang ilmu ya dok,apa kurang ilmu dok,apa kurang ilmu dok,apa kurang ilmu dok
9,video1,Cara media massa cari uang.... Memainkan keres...,cara media massa cari uang memainkan keresahan...,cara media massa cari uang main resah bingung ...,cara media massa cari uang main resah bingung ...,cara media massa cari uang main resah bingung ...


In [29]:
# Tokenizing
def tokenize_text(text):
    # Tokenisasi teks menjadi kata-kata
    tokens = word_tokenize(text)
    return tokens

In [30]:
df["text_tokenize"] = df["text_manual_replaced"].apply(lambda x: tokenize_text(x))
df.head(10)

,video_id,comment_text,text_cleaned,text_removal,text_normalized,text_manual_replaced,text_tokenize
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,"[baik, bagaimana, turut, kalian]"
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,"[betul, dok, betul, dok]"
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa,betul dok mungkin efek sa,betul dok mungkin efek sa,"[betul, dok, mungkin, efek, sa]"
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu,bpom aneh kerja kok itu,bpom aneh kerja kok itu,"[bpom, aneh, kerja, kok, itu]"
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...,benar banget anak saya saja harus sirup minum ...,benar banget anak aku saja harus sirup minum o...,"[benar, banget, anak, aku, saja, harus, sirup,..."
5,video1,Makasih ilmunya dokter semoga berkah🥰🥰🥰🥰,makasih ilmunya dokter semoga berkah,makasih ilmu dokter moga berkah,terima kasih ilmu dokter moga berkah,terima kasih ilmu dokter semoga berkah,"[terima, kasih, ilmu, dokter, semoga, berkah]"
6,video1,Dok tempra gimana? Soalnya anakku kalau demam ...,dok tempra gimana soalnya anakku kalau demam d...,dok tempra gimana soal anak kalau demam kasih ...,dok tempra bagaimana soal anak kalau demam kas...,dok tempra bagaimana soal anak kalau demam kas...,"[dok, tempra, bagaimana, soal, anak, kalau, de..."
7,video1,BIOSOLFON KIDS AMAN GAK KAK,biosolfon kids aman gak kak,biosolfon kids aman gak kak,biosolfon kids aman enggak kak,biosolfon kids aman enggak kak,"[biosolfon, kids, aman, enggak, kak]"
8,video1,apa kurang ilmu ya dok,apa kurang ilmu ya dok,apa kurang ilmu dok,apa kurang ilmu dok,apa kurang ilmu dok,"[apa, kurang, ilmu, dok]"
9,video1,Cara media massa cari uang.... Memainkan keres...,cara media massa cari uang memainkan keresahan...,cara media massa cari uang main resah bingung ...,cara media massa cari uang main resah bingung ...,cara media massa cari uang main resah bingung ...,"[cara, media, massa, cari, uang, main, resah, ..."


In [31]:
df.to_csv('../Data/2. Clean/clean.csv', index=False)